# Основы анализа данных в Python

*Алла Тамбовцева*

## Практикум 4. Доверительные интервалы


Импортируем необходимые библиотеки:
    
* `numpy`: для математических функций и работы с массивами;
* модуль `stats` из `scipy` для статистических вычислений.

In [1]:
import numpy as np
from scipy import stats as st

## Часть 1: работаем с формулами

### Задача 1: доверительный интервал для среднего

*Из независимого экзамена*

Винни-Пух в течение 150 дней фиксировал изменения количества пчёл в улье. Он уверен, что полученные наблюдения являются выборкой независимых одинаково распределённых нормальных случайных величин. Оказалось, что среднее количество пчёл равно 25000, а выборочная дисперсия равна 1300. Постройте 95%-ый доверительный интервал для математического ожидания количества пчёл в улье, используя Z-оценку, и выпишите в ответ его нижнюю границу, округлённую до целого.

### Решение 1: формулы only

Зафиксируем все параметры из условия:

* `n`: объем выборки;
* `mean`: среднее выборки;
* `s2`: выборочная дисперсия; 

In [2]:
n = 150
mean = 25000
s2 = 1300

Вспомним, как строится 95%-ный доверительный интервал для среднего (в предположении, что выборка достаточно большая и взята из нормального распределения или близкого к нему):

$$
\text{mean} - 2 \times \text{SE} < \mu < \text{mean} + 2 \times \text{SE}.
$$

Здесь $\mu$ – среднее генеральной совокупности, которое нам неизвестно и которое мы хотим оценить с помощью доверительного интервала, $\text{mean}$ – среднее, полученное по выборке, $\text{SE}$ – стандартное отклонение среднего, тоже полученное по выборке, которое вычисляется на основе стандартного отклонения самой выборки $s$ и ее объема $n$:

$$
\text{SE} = \frac{s}{\sqrt{n}}.
$$


Вычисляем стандартную ошибку:

In [3]:
SE = np.sqrt(s2) / np.sqrt(n)
print(SE)

2.943920288775949


Вычисляем границы доверительного интервала:

In [4]:
(mean - 2 * SE, mean + 2 * SE)

(24994.11215942245, 25005.88784057755)

На всякий случай вычислим границы 95%-ного доверительного интервала поточнее – вспомним, что на самом деле, если мы используем z-оценку, то есть квантиль стандартного нормального распределения, нам нужно домножать стандартную ошибку не на 2, а на 1.96:

In [5]:
(mean - 1.96 * SE, mean + 1.96 * SE)

(24994.229916234, 25005.770083766)

Если округлим полученные границы, получим интервал [24994, 25006]. Значит, с 95%-ной уверенностью мы можем утверждать, что среднее число пчел в ульях лежит в интервале от 24994 до 25006. В ответ задачи пойдет число 24994 – нам была нужна нижняя граница интервала, округленная до целого.

### Решение 2: подключаем `scipy.stats`

В модуле `stats` библиотеки `scipy` есть функция `norm.interval()` для построения доверительных интервалов с использованием z-оценок (квантилей стандартного нормального распределения) и функция `t.interval()` для построения доверительных интервалов с использованием t-оценок (квантилей распределения Стьюдента). Первая функция подходит для построения доверительных интервалов для доли или среднего в случае большой выборки, вторая функция – для построения доверительных интервалов для среднего в случае выборки любого размера. 

Плюс этих функций: они сами считают необходимое z-значение или t-значение, исходя из заданного уровня доверия и объема выборки. Минус этих функций: на вход они принимают стандартную ошибку доли или среднего, поэтому предварительно ее нужно посчитать самим.

Применим функцию для решения нашей задачи. Раз нам нужен интервал с использованием z-оценки, возьмем функцию `norm.interval()`:

In [6]:
# st – из stats, его импортировали в самом начале
# на первом месте уровень доверия
# loc: центр интервала, здесь среднее
# scale: разброс, здесь стандартная ошибка среднего

st.norm.interval(0.95, loc = mean, scale = SE)

(24994.230022260643, 25005.769977739357)

Получили те же самые границы, но более точные. Если бы использовали t-оценки, то есть квантили из распределения Стьюдента (строго говоря, их и надо использовать, просто когда выборка большая, соответствующее число степеней свободы распределения Стьюдента тоже большое, поэтому само распределение практически неотличимо от стандартного распределения), нам понадобилась бы функция `t.interval()`:

In [7]:
# df – число степеней свободы, df = n - 1 

st.t.interval(0.95, loc = mean, scale = SE, df = n - 1)

(24994.18277471534, 25005.81722528466)

Теперь еще более точные границы :) Для маленьких выборок объема менее 30 наблюдений будет актуально, потому что на маленьких объемах приближать распределение Стьюдента стандартным нормальным некорректно.

### Задача 2: доверительный интервал  для доли

*Не из независимого экзамена, тут видно*

Винни-Пух, прикинувшись тучкой, опросил 1500 пчел и выяснил, что 1020 пчел любят собирать мед на Пуховой опушке, остальные – в лесу. Постройте 90%-ный доверительный интервал для доли пчел, которые любят собирать мед на Пуховой опушке.

Вспоминаем формулу в более общем виде (для любого уровня доверия):

$$
\hat{p} - z^* \times \text{SE} < p < \hat{p} + z^* \times \text{SE}.
$$

Здесь $z^*$ – соответствующее значение квантиля стандартного нормального распределения, $p$ – доля в генеральной совокупности, которая нам неизвестна и которую мы хотим оценить с помощью доверительного интервала, $\hat{p}$ – доля, полученное по выборке, $\text{SE}$ – стандартное отклонение доли, тоже полученное по выборке, которое вычисляется на основе $\hat{p}$ и $n$:

$$
\text{SE} = \frac{\sqrt{\hat{p}(1- \hat{p})}}{\sqrt{n}}.
$$


In [8]:
p = 1020 / 1500
se_p = np.sqrt(p * (1 - p) / 1500)
st.norm.interval(0.9, loc = p, scale = se_p)

(0.6601887867507454, 0.6998112132492547)

## Часть 2: работаем с массивами

### Доверительный интервал для среднего

Представим, что у нас есть массив значений заработной платы (в тысячах рублей) сотрудников некоторой компании:

In [9]:
salary = np.array([50, 60, 90, 60, 55, 45, 50, 30, 35])

Для того, чтобы построить доверительный интервал для среднего, нам понадобится выборочное среднее и стандартная ошибка среднего. Для вычисления среднего воспользуемся функцией `mean()` из `numpy`, для вычисления стандартной ошибки – функцией `sem()` из `stats` (сокращение от *standard error of a mean*):

In [10]:
# среднее

m = np.mean(salary)
print(m)

52.77777777777778


In [11]:
# стандартная ошибка среднего

se_m = st.sem(salary)
print(se_m)

5.780181124079114


Так как выборка маленькая, мы будем пользоваться t-значениями, квантилями из распределения Стьюдента. А для этого необходимо зафиксировать число наблюдений в выборке (понадобится для числа степеней свободы):

In [12]:
# атрибут size

n = salary.size
print(n)

9


Теперь построим 95%-ный доверительный интервал для среднего значения заработной платы:

In [13]:
# loc: интервал симметричен относительно среднего
# scale: показывает средний разброс вокруг среднего в loc
# df = n - 1, число степеней свободы

st.t.interval(0.95, loc = m, scale = se_m, df = n - 1)

(39.4486562044095, 66.10689935114605)

Так как выборка очень маленькая, интервал получился широким. Это неприятно, но в данном случае ожидамо. С 95%-ной уверенностью можно утверждать, что истинное среднее значение заработной платы (не по опрошенным сотрудникам, а по всем сотрудникам компании) лежит в интервале от 39 до 66 тысяч рублей.

Если бы мы построили 99%-ный доверительный интервал, он оказался бы еще шире:

In [14]:
st.t.interval(0.99, loc = m, scale = se_m, df = n - 1)

(33.38303126123029, 72.17252429432526)

### Доверительный интервал для доли

Представим себе, что у нас есть массив из 0 и 1, где значением 1 закодированы ответы «да», а значением 0 – ответы «нет»:

In [15]:
yes_no = np.array([0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0,
                  0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 
                  0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0,
                  0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0])

Мы уже выяснили, что для построения доверительного интервала для доли нам нужно знать значение $\hat{p}$ и $\text{SE}$.

В нашем случае $\hat{p}$ – это доля единиц. Поэтому для вычисления доли мы можем воспользоваться функцией для нахождения среднего, так как:

* среднее – это сумма элементов, деленная на общее число элементов;
* доля – это число единиц, деленное на общее число элементов;
* при суммировании нули не считаются, поэтому результаты выше эквивалентны.

Итак, доля:

In [16]:
p = np.mean(yes_no)
print(p)

0.4


По аналогичным причинам стандартная ошибка доли в случае бинарного массива будет совпадать со стандартной ошибкой среднего, поэтому SE здесь:

In [17]:
se_p = st.sem(yes_no)
print(se_p)

0.05511782546095504


Теперь построим 95%-ный доверительный интервал для доли:

In [18]:
st.norm.interval(0.95, loc = p, scale = se_p)

(0.2919710471903633, 0.5080289528096367)

С 95%-ной уверенностью можно утверждать, что истинная доля ответивших «да» (доля в генеральной совокупности, среди всех людей) лежит в интервале от 0.29 до 0.51.

Конечно, далеко не всегда данные представлены в виде готовых удобных массивов из 0 и 1. Но, что удобно, любой массив можно свести к бинарному, а как считать доли для бинарных, мы уже знаем.

Предположим, у нас есть массив из текстовых значений `yes` и `no`:

In [19]:
answers = np.array(["yes", "no", "yes", "no", "yes", "yes", "yes"])

Как определить долю ответов yes? Первый шаг: проверяем равенство значению `yes` для всех элементов массива:

In [20]:
answers == "yes"

array([ True, False,  True, False,  True,  True,  True])

Получили набор из логических значений `True` и `False`. Python умеет автоматически заменять `True` на 1, а `False` на 0, поэтому к таким массивам (булевы массивы, так как логический тип данных еще называется булевым по имени логика Дж.Буля) можно применять те же функции суммирования и нахождения среднего. Отсюда второй шаг: считаем среднее по полученному массиву:

In [21]:
# доля ответов yes

np.mean(answers == "yes")

0.7142857142857143

А дальше можно действовать по уже знакомому плану – вычислить стандартную ошибку и границы интервала.

In [22]:
# например, 95%-ный

st.norm.interval(0.95, loc = np.mean(answers == "yes"), 
                scale = st.sem(answers == "yes"))

(0.35281391089095426, 1.0757575176804743)

Чисто технически можно получить какой-то нереалистично широкий доверительный интервал, границы которого выходят за разумные пределы, если стандартная ошибка оказалась очень большой. В данном случае это и произошло: мы построили доверительный интервал для доли на основе очень маленькой выборки. Вообще доля не может выходить за рамки отрезка от 0 до 1, а верхняя граница нашего интервала здесь 1.08. При интерпретации мы можем зацензурировать полученный интервал и сказать, что с 95%-ной уверенностью доля ответивших «да» лежит в интервале от 0.35 до 1 или от 35% до 100% (но все равно интервал неинформативный, слишком широкий, выборка в 7 наблюдений – это несерьезно).

## Часть 3: работаем с реальными данными

Импортируем `pandas`:

In [23]:
import pandas as pd

Загрузим данные по видеоиграм из файла `c.xlsx` (мы с ним уже работали):

In [24]:
games = pd.read_excel("c.xlsx")

Посмотрим на число пропущенных значений в каждом столбце:

In [25]:
games.isnull().sum()

Unnamed: 0            0
Name                  2
Platform              0
Year_of_Release     269
Genre                 2
Publisher            54
NA_Sales              0
EU_Sales              0
JP_Sales              0
Other_Sales           0
Global_Sales          0
Critic_Score       8582
Critic_Count       8582
User_Score         6704
User_Count         9129
Developer          6623
Rating             6769
dtype: int64

Попробуем построить 95%-ный доверительный интервал для среднего значения `Critic_Score` (рейтинг команды Metacritic). 
Вычислим среднее:

In [26]:
games["Critic_Score"].mean()

68.96767850559173

Вычислим стандартную ошибку среднего:

In [27]:
st.sem(games["Critic_Score"]) # проблема!

nan

Из-за пропусков стандартная ошибка не вычисляется. Тут два пути: либо заполнить пропуски (например, средним или медианой), либо удалить их. Давайте пойдем по второму пути, но сделаем это осторожно. Мы не будем удалять строки с пропущенными значениями вообще, а удалим только те строки, где есть пропуски в столбце `Critic_Score`:

In [28]:
games.dropna(subset = ["Critic_Score"], inplace = True)

Проверим, что осталось, и сколько пропусков в датафрейме сейчас:

In [29]:
games.isnull().sum()

Unnamed: 0            0
Name                  0
Platform              0
Year_of_Release     154
Genre                 0
Publisher             4
NA_Sales              0
EU_Sales              0
JP_Sales              0
Other_Sales           0
Global_Sales          0
Critic_Score          0
Critic_Count          0
User_Score           38
User_Count         1120
Developer             6
Rating               83
dtype: int64

Попробуем снова посчитать среднее и стандартную ошибку среднего по `Critic_Score`:

In [30]:
games["Critic_Score"].mean()  # не изменилось, mean() игнорирует пропуски

68.96767850559173

In [31]:
st.sem(games["Critic_Score"]) # вычисляется!

0.15451599023343576

Теперь осталось вычислить 95%-ный доверительный интервал (проверьте, чего нам для него еще не хватает! 

In [32]:
# не хватает n для df

st.t.interval(0.95, 
              loc = games["Critic_Score"].mean(), 
              scale = st.sem(games["Critic_Score"]), 
              df = games["Critic_Score"].size) 

(68.66478767532462, 69.27056933585885)